In [99]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib import urlopen
import re
import cookielib
import urllib2
import psycopg2

In [2]:
url = 'http://www.nytimes.com/pages/world/index.html?action=click&pgtype=Homepage&region=TopBar&module=HPMiniNav&contentCollection=World&WT.nav=page'
soup = BeautifulSoup(urlopen(url).read(), 'html.parser')
results = soup.find_all('div', attrs = {'class': 'story'})

In [3]:
# initialize lists
titles = []
dates = []
links = []
full_texts = []
authors = []
sections = []

# scrape results into lists
for x in results:
    
    # get link
    link = x.find('a')['href']
    # set regex to eliminate interactive features
    match = re.search('^http://www.nytimes.com/20', link)
    if match:
        links.append(link)
    
# resoup it
for link in links:
    all_p = ''
    cj = cookielib.CookieJar()
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
    new_soup = BeautifulSoup(opener.open(link).read(), 'html.parser')
    # get the article content
    body = new_soup.find_all('p', attrs = {'class': 'story-body-text story-content'})
    for p in body:
        new_p = p.text.strip()
        all_p = all_p + new_p
    full_texts.append(all_p)
    
    # get titles
    title = new_soup.find('meta', attrs = {'property': 'og:title'})['content']
    titles.append(title)
    
    # get authors
    author = new_soup.find('meta', attrs = {'name': 'author'})['content']
    authors.append(author)
    
    # get sections
    section = new_soup.find('meta', attrs = {'name': 'CG'})['content']
    sections.append(section)
    
    # get dates
    date = new_soup.find('meta', attrs = {'name': 'pdate'})['content']
    dates.append(date)
    

In [9]:
# make world dataframe
world = pd.DataFrame(columns = ['link','date','body','title','author','section'])
world['link'] = links
world['date'] = dates
world['body'] = full_texts
world['title'] = titles
world['author'] = authors
world['section'] = sections

In [10]:
world.head()

,link,date,body,title,author,section
0,http://www.nytimes.com/2017/01/27/world/americ...,20170127,MEXICO CITY — Amid the diplomatic showdown ove...,Trump and Mexican President Speak by Phone Ami...,Kirk Semple,world
1,http://www.nytimes.com/2017/01/27/world/americ...,20170127,"TIJUANA, Mexico — Even before President Trump ...","As Migrants Strain Border Towns, Pressure Buil...",Kirk Semple,world
2,http://www.nytimes.com/2017/01/26/world/africa...,20170126,"DAKAR, Senegal — The clinic, tucked discreetly...",Clinics for World’s Vulnerable Brace for Trump...,"Dionne Searcey, Norimitsu Onishi and Somini Se...",world
3,http://www.nytimes.com/2017/01/27/world/africa...,20170127,"MOGADISHU, Somalia — A suicide bomber from the...",Shabab Militants Attack Kenyan Military Base i...,Hussein Mohamed,world
4,http://www.nytimes.com/2017/01/27/world/austra...,20170127,HONG KONG — You might call it a Noah’s Ark for...,"As Climate Change Accelerates, Floating Cities...",Mike Ives,world


In [16]:
world['section'].value_counts()

world          33
opinion         3
us              1
automobiles     1
Name: section, dtype: int64

In [54]:
world.to_csv('/Users/teresaborcuch/Desktop/nyt_world.csv', encoding = 'utf8')

In [11]:
# now for opinion
url = 'http://www.nytimes.com/pages/opinion/index.html?action=click&region=TopBar&pgtype=SectionFront&module=SectionsNav&version=BrowseTree&contentCollection=Opinion&t=qry842'
soup = BeautifulSoup(urlopen(url).read(), 'html.parser')
results = soup.find_all('div', attrs = {'class': 'story'})

In [12]:
# initialize lists
titles = []
dates = []
links = []
full_texts = []
authors = []
sections = []

# scrape results into lists
for x in results:
    
    # get link
    link = x.find('a')['href']
    # set regex to eliminate interactive features
    match = re.search('^http://www.nytimes.com/20', link)
    if match:
        links.append(link)
    
# resoup it
for link in links:
    all_p = ''
    cj = cookielib.CookieJar()
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
    new_soup = BeautifulSoup(opener.open(link).read(), 'html.parser')
    # get the article content
    body = new_soup.find_all('p', attrs = {'class': 'story-body-text story-content'})
    for p in body:
        new_p = p.text.strip()
        all_p = all_p + new_p
    full_texts.append(all_p)
    
    # get titles
    title = new_soup.find('meta', attrs = {'property': 'og:title'})['content']
    titles.append(title)
    
    # get authors
    author = new_soup.find('meta', attrs = {'name': 'author'})['content']
    authors.append(author)
    
    # get sections
    section = new_soup.find('meta', attrs = {'name': 'CG'})['content']
    sections.append(section)
    
    # get dates
    date = new_soup.find('meta', attrs = {'name': 'pdate'})['content']
    dates.append(date)
    

In [13]:
# make opinion dataframe
op = pd.DataFrame(columns = ['link','date','body','title','author','section'])
op['link'] = links
op['date'] = dates
op['body'] = full_texts
op['title'] = titles
op['author'] = authors
op['section'] = sections

In [14]:
op.head()

,link,date,body,title,author,section
0,http://www.nytimes.com/2017/01/26/opinion/dona...,20170126,"Less than a week into the job, President Trump...",Donald Trump’s Mexico Tantrum,The Editorial Board,opinion
1,http://www.nytimes.com/2017/01/27/opinion/the-...,20170127,This is a column directed at high school and c...,The Politics of Cowardice,David Brooks,opinion
2,http://www.nytimes.com/2017/01/27/opinion/the-...,20170127,Donald Trump is an ahistorical man. He knows n...,The Closing of Trump’s America,Roger Cohen,opinion
3,http://www.nytimes.com/2017/01/27/opinion/maki...,20170127,Donald Trump will break most of his campaign p...,Making the Rust Belt Rustier,Paul Krugman,opinion
4,http://www.nytimes.com/2017/01/27/opinion/park...,20170127,"It started at the inauguration, when the unifo...",Park Rangers to the Rescue,Timothy Egan,opinion


In [53]:
op.to_csv('/Users/teresaborcuch/Desktop/nyt_opinion.csv', encoding = 'utf8')

In [15]:
op['section'].value_counts()

opinion    36
insider     1
Name: section, dtype: int64

In [17]:
# now for politics
url = 'http://www.nytimes.com/pages/politics/index.html?action=click&region=TopBar&pgtype=SectionFront&module=SectionsNav&version=BrowseTree&contentCollection=Politics&t=qry822'
soup = BeautifulSoup(urlopen(url).read(), 'html.parser')
results = soup.find_all('div', attrs = {'class': 'story'})

In [18]:
# initialize lists
titles = []
dates = []
links = []
full_texts = []
authors = []
sections = []

# scrape results into lists
for x in results:
    
    # get link
    link = x.find('a')['href']
    # set regex to eliminate interactive features
    match = re.search('^http://www.nytimes.com/20', link)
    if match:
        links.append(link)
    
# resoup it
for link in links:
    all_p = ''
    cj = cookielib.CookieJar()
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
    new_soup = BeautifulSoup(opener.open(link).read(), 'html.parser')
    # get the article content
    body = new_soup.find_all('p', attrs = {'class': 'story-body-text story-content'})
    for p in body:
        new_p = p.text.strip()
        all_p = all_p + new_p
    full_texts.append(all_p)
    
    # get titles
    title = new_soup.find('meta', attrs = {'property': 'og:title'})['content']
    titles.append(title)
    
    # get authors
    author = new_soup.find('meta', attrs = {'name': 'author'})['content']
    authors.append(author)
    
    # get sections
    section = new_soup.find('meta', attrs = {'name': 'CG'})['content']
    sections.append(section)
    
    # get dates
    date = new_soup.find('meta', attrs = {'name': 'pdate'})['content']
    dates.append(date)

In [19]:
# make politics dataframe
politics = pd.DataFrame(columns = ['link','date','body','title','author','section'])
politics['link'] = links
politics['date'] = dates
politics['body'] = full_texts
politics['title'] = titles
politics['author'] = authors
politics['section'] = sections

In [20]:
politics.head()

,link,date,body,title,author,section
0,http://www.nytimes.com/2017/01/27/us/politics/...,20170127,Donald J. Trump spent his first week in office...,Who Is Registered to Vote in Two States? Some ...,Erin McCann,us
1,http://www.nytimes.com/2017/01/27/us/politics/...,20170127,WASHINGTON — President Trump used Twitter on F...,A Dubious Vote-Fraud Claim Gets the Trump Seal...,Michael D. Shear,us
2,http://www.nytimes.com/2017/01/27/us/politics/...,20170127,Abortion opponents gathered on Friday in Washi...,Pence Tells Anti-Abortion Marchers That ‘Life ...,"Jeremy W. Peters, Yamiche Alcindor and Noah We...",us
3,http://www.nytimes.com/2017/01/27/us/politics/...,20170127,■ President Trump would not say whether he fav...,Trump Avoids Answering Whether He Will Lift Sa...,"Michael D. Shear, Helene Cooper and Jonathan W...",us
4,http://www.nytimes.com/2017/01/26/us/politics/...,20170126,WASHINGTON — President Trump spoke by telephon...,Trump Called National Park Chief Over Twitter ...,Michael D. Shear and Maggie Haberman,us


In [52]:
politics.to_csv('/Users/teresaborcuch/Desktop/nyt_politics.csv', encoding = 'utf8')

In [21]:
politics['section'].value_counts()

us            8
world         3
opinion       2
business      1
realestate    1
Name: section, dtype: int64

In [22]:
# now for business
url = 'http://www.nytimes.com/pages/business/index.html?action=click&region=TopBar&pgtype=SectionFront&module=SectionsNav&version=BrowseTree&contentCollection=Business&t=qry542'
soup = BeautifulSoup(urlopen(url).read(), 'html.parser')
results = soup.find_all('div', attrs = {'class': 'story'})

In [23]:
# initialize lists
titles = []
dates = []
links = []
full_texts = []
authors = []
sections = []

# scrape results into lists
for x in results:
    
    # get link
    link = x.find('a')['href']
    # set regex to eliminate interactive features
    match = re.search('^http://www.nytimes.com/20', link)
    if match:
        links.append(link)
    
# resoup it
for link in links:
    all_p = ''
    cj = cookielib.CookieJar()
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
    new_soup = BeautifulSoup(opener.open(link).read(), 'html.parser')
    # get the article content
    body = new_soup.find_all('p', attrs = {'class': 'story-body-text story-content'})
    for p in body:
        new_p = p.text.strip()
        all_p = all_p + new_p
    full_texts.append(all_p)
    
    # get titles
    title = new_soup.find('meta', attrs = {'property': 'og:title'})['content']
    titles.append(title)
    
    # get authors
    author = new_soup.find('meta', attrs = {'name': 'author'})['content']
    authors.append(author)
    
    # get sections
    section = new_soup.find('meta', attrs = {'name': 'CG'})['content']
    sections.append(section)
    
    # get dates
    date = new_soup.find('meta', attrs = {'name': 'pdate'})['content']
    dates.append(date)

In [24]:
# make business dataframe
business = pd.DataFrame(columns = ['link','date','body','title','author','section'])
business['link'] = links
business['date'] = dates
business['body'] = full_texts
business['title'] = titles
business['author'] = authors
business['section'] = sections

In [25]:
business.head()

,link,date,body,title,author,section
0,http://www.nytimes.com/2017/01/27/business/med...,20170127,NBC is looking to place its new star Megyn Kel...,"Megyn Kelly May Get NBC Morning Show, Shaking ...",John Koblin and Michael M. Grynbaum,business
1,http://www.nytimes.com/2017/01/27/business/eco...,20170127,The American economy finished 2016 on a soft n...,Modest U.S. Growth at Year’s End May Reframe P...,Nelson D. Schwartz,business
2,http://www.nytimes.com/2017/01/27/business/vol...,20170127,German prosecutors said for the first time on ...,Ex-Volkswagen Chief Investigated by Germany in...,Jack Ewing,business
3,http://www.nytimes.com/2017/01/26/business/eco...,20170126,The idea of a broad tax on importers is sudden...,Counting the Winners and Losers From an Import...,"Keith Bradsher, Rachel Abrams and Bill Vlasic",business
4,http://www.nytimes.com/2017/01/26/us/politics/...,20170126,WASHINGTON — President Trump campaigned on swe...,What Trump Can and Can’t Do to Dismantle Obama...,Coral Davenport,us


In [51]:
business.to_csv('/Users/teresaborcuch/Desktop/nyt_business.csv', encoding = 'utf8')

In [27]:
business['section'].value_counts()

business       17
technology      3
your-money      2
us              2
jobs            2
opinion         2
world           1
automobiles     1
Name: section, dtype: int64

In [28]:
# science
url = 'https://www.nytimes.com/section/science?action=click&region=TopBar&pgtype=SectionFront&module=SectionsNav&version=BrowseTree&contentCollection=Science&t=qry873'
soup = BeautifulSoup(urlopen(url).read(), 'html.parser')
results = soup.find_all('div', attrs = {'class': 'story'})

In [29]:
# initialize lists
titles = []
dates = []
links = []
full_texts = []
authors = []
sections = []

# scrape results into lists
for x in results:
    
    # get link
    link = x.find('a')['href']
    # set regex to eliminate interactive features
    match = re.search('^http://www.nytimes.com/20', link)
    if match:
        links.append(link)
    
# resoup it
for link in links:
    all_p = ''
    cj = cookielib.CookieJar()
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
    new_soup = BeautifulSoup(opener.open(link).read().decode('utf-8','ignore'), 'html.parser')
    # get the article content
    body = new_soup.find_all('p', attrs = {'class': 'story-body-text story-content'})
    for p in body:
        new_p = p.text.strip()
        all_p = all_p + new_p
    full_texts.append(all_p)
    
    # get titles
    title = new_soup.find('meta', attrs = {'property': 'og:title'})['content']
    titles.append(title)
    
    # get authors
    author = new_soup.find('meta', attrs = {'name': 'author'})['content']
    authors.append(author)
    
    # get sections
    section = new_soup.find('meta', attrs = {'name': 'CG'})['content']
    sections.append(section)
    
    # get dates
    date = new_soup.find('meta', attrs = {'name': 'pdate'})['content']
    dates.append(date)

In [30]:
# make science dataframe
science = pd.DataFrame(columns = ['link','date','body','title','author','section'])
science['link'] = links
science['date'] = dates
science['body'] = full_texts
science['title'] = titles
science['author'] = authors
science['section'] = sections

In [31]:
science.head()

,link,date,body,title,author,section


In [62]:
nyt_data = pd.concat([business,op,world,politics], join = 'outer', ignore_index = True)

In [63]:
nyt_data

,link,date,body,title,author,section
0,http://www.nytimes.com/2017/01/27/business/med...,20170127,NBC is looking to place its new star Megyn Kel...,"Megyn Kelly May Get NBC Morning Show, Shaking ...",John Koblin and Michael M. Grynbaum,business
1,http://www.nytimes.com/2017/01/27/business/eco...,20170127,The American economy finished 2016 on a soft n...,Modest U.S. Growth at Year’s End May Reframe P...,Nelson D. Schwartz,business
2,http://www.nytimes.com/2017/01/27/business/vol...,20170127,German prosecutors said for the first time on ...,Ex-Volkswagen Chief Investigated by Germany in...,Jack Ewing,business
3,http://www.nytimes.com/2017/01/26/business/eco...,20170126,The idea of a broad tax on importers is sudden...,Counting the Winners and Losers From an Import...,"Keith Bradsher, Rachel Abrams and Bill Vlasic",business
4,http://www.nytimes.com/2017/01/26/us/politics/...,20170126,WASHINGTON — President Trump campaigned on swe...,What Trump Can and Can’t Do to Dismantle Obama...,Coral Davenport,us
5,http://www.nytimes.com/2017/01/26/business/eco...,20170126,Will President Trump be able to make Mexico pa...,Making Mexico Pay for Border Wall: Ideas Aboun...,Eduardo Porter,business
6,http://www.nytimes.com/2017/01/26/business/med...,20170126,WASHINGTON — Just days after President Trump s...,Trump Strategist Stephen Bannon Says Media Sho...,Michael M. Grynbaum,business
7,http://www.nytimes.com/2017/01/27/business/dea...,20170127,TOKYO — Ill-fated investments in nuclear power...,"Toshiba, Desperate for Cash After Scandal, Wil...",Jonathan Soble,business
8,http://www.nytimes.com/2017/01/26/automobiles/...,20170126,BEIJING — On a windswept lot near Beijing’s ma...,China’s Answer to Tesla Is Hopeful Entrant to ...,Michael Schuman,automobiles
9,http://www.nytimes.com/2017/01/27/business/inn...,20170127,Sean Sweeney opened the door to the refurbishe...,Innovations Threaten the Neighborhood Bike Shop,Jim Rendon,business


In [64]:
nyt_data.to_csv('/Users/teresaborcuch/Desktop/nyt_data.csv', encoding = 'utf8')

In [35]:
def get_lists(results):
    # initialize lists
    titles = []
    dates = []
    links = []
    full_texts = []
    authors = []
    sections = []

    # scrape results into lists
    for x in results:
        # get link
        link = x.find('a')['href']
        # set regex to eliminate interactive features
        match = re.search('^http://www.nytimes.com/20', link)
        if match:
            links.append(link)

    # resoup it
    for link in links:
        all_p = ''
        cj = cookielib.CookieJar()
        opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
        new_soup = BeautifulSoup(opener.open(link).read(), 'html.parser')
        # get the article content
        body = new_soup.find_all('p', attrs = {'class': 'story-body-text story-content'})
        for p in body:
            new_p = p.text.strip()
            all_p = all_p + new_p
        full_texts.append(all_p)

        # get titles
        title = new_soup.find('meta', attrs = {'property': 'og:title'})['content']
        titles.append(title)

        # get authors
        author = new_soup.find('meta', attrs = {'name': 'author'})['content']
        authors.append(author)

        # get sections
        section = new_soup.find('meta', attrs = {'name': 'CG'})['content']
        sections.append(section)

        # get dates
        date = new_soup.find('meta', attrs = {'name': 'pdate'})['content']
        dates.append(date)
        
        return links, dates, full_texts, titles, authors, sections
    
def make_df():
    # make science dataframe
    name = pd.DataFrame(columns = ['link','date','body','title','author','section'])
    name['link'] = links
    name['date'] = dates
    name['body'] = full_texts
    name['title'] = titles
    name['author'] = authors
    name['section'] = sections

In [70]:
url = 'https://www.nytimes.com/section/world'

In [71]:
results = get_soup(url)

[<div class="story-body">\n<h2 class="headline">\n<a data-rref="" href="http://www.nytimes.com/2015/12/15/world/middleeast/islamic-state-gains-strength-in-yemen-rivaling-al-qaeda.html">Islamic State Gains Strength in Yemen, Challenging Al Qaeda</a>\n</h2>\n<div class="thumb">\n<a aria-hidden="true" data-rref="" href="http://www.nytimes.com/2015/12/15/world/middleeast/islamic-state-gains-strength-in-yemen-rivaling-al-qaeda.html" role="presentation" tabindex="-1">\n<img alt="" role="presentation" src="https://static01.nyt.com/images/2015/12/15/world/middleeast/15Yemen-web1/15Yemen-web1-thumbStandard.jpg"/>\n</a>\n</div>\n<p class="summary">An affiliate has taken root in the chaos of a civil war, apparently determined to distinguish itself as Yemen\u2019s most disruptive and brutal force.</p>\n<p class="byline">\n<span class="freshness"><time data-utc-timestamp="1450108927" datetime="1450108927"></time></span>\n<span class="divider"></span>\n<span class="author" itemprop="author">By SHUAI

In [45]:
def get_data(df_name, url):
    soup = get_soup(url)
    get_lists(soup)
    df_name = make_df()
    return df_name  

In [46]:
url = 'https://www.nytimes.com/section/health?action=click&contentCollection=Health&module=SectionsNav&pgtype=article&region=TopBar&version=BrowseTree'
get_data('health', url)

,link,date,body,title,author,section


In [43]:
tech = make_df()

In [48]:
results = get_soup(url)

In [49]:
results

[]

# TRYING TO MAKE A FUNCTION, ALSO NEW URLS

In [69]:
def get_soup(url):
    soup = BeautifulSoup(urlopen(url).read().decode('utf-8'), 'html.parser')
    results = soup.find_all('div', attrs = {'class': 'story-body'})
    return results

In [109]:
def make_data(results):
    
    # initialize lists
    titles = []
    dates = []
    links = []
    full_texts = []
    authors = []
    sections = []

    # scrape results into lists
    for x in results:
        # get link
        link = x.find('a')['href']
        # set regex to eliminate interactive features
        match = re.search('^http://www.nytimes.com/20', link)
        if match:
            links.append(link)

    # resoup it
    for link in links:
        all_p = ''
        cj = cookielib.CookieJar()
        opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
        new_soup = BeautifulSoup(opener.open(link).read(), 'html.parser')
        # get the article content
        body = new_soup.find_all('p', attrs = {'class': 'story-body-text story-content'})
        for p in body:
            new_p = p.text.strip()
            all_p = all_p + new_p
        full_texts.append(all_p)

        # get titles
        title = new_soup.find('meta', attrs = {'property': 'og:title'})['content']
        titles.append(title)

        # get authors
        author = new_soup.find('meta', attrs = {'name': 'author'})['content']
        authors.append(author)

        # get sections
        section = new_soup.find('meta', attrs = {'name': 'CG'})['content']
        sections.append(section)

        # get dates
        date = new_soup.find('meta', attrs = {'name': 'pdate'})['content']
        dates.append(date)
        
    data_dict = {
        'title': titles, 'link': links, 'author': authors, 'body': full_texts, 
        'section': sections, 'date': dates
    }
    return data_dict
    

In [110]:
data_dict = make_data(results)

In [111]:
def make_df(data_dict):
    df = pd.DataFrame(data_dict)
    return df

In [112]:
df = make_df(data_dict)

# CONNECT TO DB

In [102]:
from sqlalchemy import create_engine

In [113]:
engine = create_engine('postgresql://teresaborcuch@localhost:5433/capstone')

In [114]:
#conn = psycopg2.connect('host=localhost port=5433 dbname=capstone user=teresaborcuch')
df.to_sql('nytimes', engine, if_exists = 'append', index = False)

In [128]:
def get(url):
    #request = urllib2.Request(url)
    response = urllib2.urlopen(url)
    soup = BeautifulSoup(response.read().decode('utf-8', 'ignore'), 'html.parser')




    #soup = BeautifulSoup(urlopen(url).read().decode('utf-8', 'ignore'), 'html.parser')
    results = soup.find_all('div')
    return results

In [130]:
sk = get("http://www.smittenkitchen.com")

In [131]:
sk

[<div class="hfeed site" id="page">\n<a class="skip-link screen-reader-text" href="#content">Skip to content</a>\n<header class="site-header" id="masthead" role="banner">\n<div id="masthead-wrapper">\n<div class="site-branding">\n<a href="https://smittenkitchen.com/" rel="home"><?xml version="1.0" encoding="utf-8"?>\n<svg class="smittenkitchen-logo" style="enable-background:new 0 0 74 62;" viewbox="0 0 74 62" x="0px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" y="0px">\n<style type="text/css">\n\t.st0{fill:#6F7997;}\n</style>\n<g class="logo-text">\n<path class="st0" d="M1.9,59.6c0.2,0.3,0.4,0.5,0.5,0.6c0.2,0.1,0.4,0.2,0.6,0.2c0.3,0,0.5-0.1,0.7-0.2C3.9,60,4,59.8,4,59.6\n\t\t\tc0-0.2-0.1-0.3-0.2-0.4c-0.1-0.1-0.4-0.3-0.9-0.4c-0.6-0.2-1-0.4-1.3-0.6c-0.3-0.3-0.5-0.7-0.5-1.1c0-0.5,0.2-0.8,0.5-1.1\n\t\t\tc0.3-0.3,0.8-0.5,1.4-0.5c0.3,0,0.6,0.1,0.9,0.2c0.3,0.1,0.5,0.2,0.6,0.4c0.1,0.1,0.2,0.3,0.3,0.5L3.9,57\n\t\t\tc-0.2-0.2-0.3-0.4-0.5-0.5c